In [96]:
import pandas as pd
import numpy as np
import string as st
import sys 

In [97]:
## Balance Sheet
bs_all = pd.read_excel('data/all/ALL_BS_TOTAL.xlsx')
bs_all = bs_all.iloc[2:,1:].reset_index(drop=True)
bs_all.columns = ['ALL'] + list(bs_all.iloc[0])[1:]
bs_all = bs_all.iloc[1:].reset_index(drop=True)
## Income Statement
is_all = pd.read_excel('data/all/ALL_IS_TOTAL.xlsx')
is_all = is_all.iloc[2:,1:].reset_index(drop=True)
is_all.columns = ['ALL'] + list(is_all.iloc[0])[1:]
is_all = is_all.iloc[1:].reset_index(drop=True)
## Dataframe
df_all = pd.concat([bs_all,is_all])
df_all = df_all.dropna().reset_index(drop=True)
df_all = df_all.reset_index()
df_all.columns = ['n'] + list(df_all.columns)[1:]
array = [ list(np.repeat('ALL',len(df_all))) , list(np.repeat('TOTAL',len(df_all)))]
tuples = list(zip(*array))
index = pd.MultiIndex.from_tuples(tuples, names=["Sector", "Size"])
df_all.index = index
df_all.columns = ['n','indicator']+list(df_all.columns)[2:]
datum_all = pd.DataFrame()
datum_all = pd.concat([datum_all,df_all])
### FX Risk
risk_all = pd.DataFrame()
key_page = ['2009-2014','2015-2021']
for page in key_page:
    df_risk = pd.read_excel('data/all/ALL_RISK_TOTAL.xlsx', sheet_name = page)
    df_risk = df_risk.iloc[7:,1:].reset_index(drop=True)
    array = [  list(df_risk.iloc[1]),list(df_risk.iloc[0].ffill())]
    tuples = list(zip(*array))
    index = pd.MultiIndex.from_tuples(tuples)
    df_risk.columns = index
    df_risk.index = list(df_risk.iloc[0:,0])
    df_risk = df_risk.loc['-FX']
    df_risk = df_risk.iloc[0:,2:]
    df_risk_1 = pd.DataFrame()
    for xx in list(df_risk.columns):
        c =  pd.DataFrame({
            'term' : xx[0].strip(),
            'year' : xx[1],
            'amount' : df_risk[xx].sum()
        }, index=[1])
        df_risk_1 = pd.concat([df_risk_1,c])
    risk_all = pd.concat([risk_all,df_risk_1])
risk_all = risk_all.pivot(index='term', columns = 'year')['amount'].reset_index()
array = [ list(np.repeat('ALL',len(risk_all))) , list(np.repeat('TOTAL',len(risk_all)))]
tuples = list(zip(*array))
index = pd.MultiIndex.from_tuples(tuples, names=["Sector", "Size"])
risk_all.index = index
datum_risk = pd.DataFrame()
datum_risk = pd.concat([datum_risk,risk_all])

In [98]:
# Creating a key for loops
key_sektor = st.ascii_uppercase
key_sektor = key_sektor[0:10]+key_sektor[11:14]+key_sektor[15:19]

In [99]:
for sektor in key_sektor:
    ## Balance Sheet
    bs_all = pd.read_excel(f'data/balance_sheet/1digit/total/{sektor}_BS_TOTAL.xlsx')
    key = bs_all.iloc[0,0]
    bs_all = bs_all.iloc[2:,1:].reset_index(drop=True)
    bs_all.columns = [key] + list(bs_all.iloc[0])[1:]
    bs_all = bs_all.iloc[1:].reset_index(drop=True)
    ## Income Statement
    is_all = pd.read_excel(f'data/income_statement/1digit/total/{sektor}_IS_TOTAL.xlsx')
    is_all = is_all.iloc[2:,1:].reset_index(drop=True)
    is_all.columns = [key] + list(is_all.iloc[0])[1:]
    is_all = is_all.iloc[1:].reset_index(drop=True)
    ## Dataframe
    df_all = pd.concat([bs_all,is_all])
    df_all = df_all.dropna().reset_index(drop=True)
    df_all = df_all.reset_index()
    df_all.columns = ['n'] + list(df_all.columns)[1:]
    array = [ list(np.repeat(key,len(df_all))) , list(np.repeat('TOTAL',len(df_all)))]
    tuples = list(zip(*array))
    index = pd.MultiIndex.from_tuples(tuples, names=["Sector", "Size"])
    df_all.index = index
    df_all.columns = ['n','indicator']+list(df_all.columns)[2:]
    datum_all = pd.concat([datum_all, df_all])

In [100]:
key_size = ['LARGE','MEDIUM','SMALL','MICRO']

In [101]:
for sektor in key_sektor:
    for size in key_size:
        ## Balance Sheet
        bs_all = pd.read_excel(f'data/balance_sheet/1digit/size/{sektor}_BS_{size}.xlsx')
        key = bs_all.iloc[0,0]
        bs_all = bs_all.iloc[2:,1:].reset_index(drop=True)
        bs_all.columns = [key] + list(bs_all.iloc[0])[1:]
        bs_all = bs_all.iloc[1:].reset_index(drop=True)
        ## Income Statement
        is_all = pd.read_excel(f'data/income_statement/1digit/size/{sektor}_IS_{size}.xlsx')
        is_all = is_all.iloc[2:,1:].reset_index(drop=True)
        is_all.columns = [key] + list(is_all.iloc[0])[1:]
        is_all = is_all.iloc[1:].reset_index(drop=True)
        ## Dataframe
        df_all = pd.concat([bs_all,is_all])
        df_all = df_all.dropna().reset_index(drop=True)
        df_all = df_all.reset_index()
        df_all.columns = ['n'] + list(df_all.columns)[1:]
        array = [ list(np.repeat(key,len(df_all))) , list(np.repeat(size,len(df_all)))]
        tuples = list(zip(*array))
        index = pd.MultiIndex.from_tuples(tuples, names=["Sector", "Size"])
        df_all.index = index
        df_all.columns = ['n','indicator']+list(df_all.columns)[2:]
        datum_all = pd.concat([datum_all, df_all])

In [102]:
for size in key_size:
    df = datum_all.swaplevel().loc[size].groupby(['n','indicator']).sum().reset_index()
    array = [ list(np.repeat('ALL',len(df))) , list(np.repeat(size,len(df)))]
    tuples = list(zip(*array))
    index = pd.MultiIndex.from_tuples(tuples, names=["Sector", "Size"])
    df.index = index
    datum_all = pd.concat([datum_all, df])

In [103]:
datum_all.to_excel('data.xlsx')

In [104]:
datum_risk.to_excel('risk.xlsx')